In [2]:
import psycopg2
from psycopg2 import OperationalError

# Configurazione del database
DB_CONFIG = {
    "dbname": "Movies_DB",
    "user": "postgres",
    "password": "Cetriolo20!",
    "host": "localhost",
    "port": "5432",
}

def create_connection():
    """Crea una connessione al database PostgreSQL."""
    try:
        print("Tentativo di connessione al database...")
        conn = psycopg2.connect(**DB_CONFIG)
        print("Connessione al database avvenuta con successo!")
        return conn
    except OperationalError as e:
        print(f"Errore durante la connessione al database: {e}")
        return None

# Test della connessione
if __name__ == "__main__":
    connection = create_connection()
    if connection:
        connection.close()
        print("Connessione chiusa correttamente.")


Tentativo di connessione al database...
Connessione al database avvenuta con successo!
Connessione chiusa correttamente.


In [26]:
import psycopg2
from psycopg2 import sql

# Configurazione del database
DB_CONFIG = {
    "dbname": "Movies_DB",
    "user": "postgres",
    "password": "Cetriolo20!",
    "host": "localhost",
    "port": "5432",
}

# Elenco delle tabelle da svuotare (in ordine per evitare errori di FK)
TABLES = [
    "actors",
    "countries",
    "crew",
    "genres",
    "languages",
    "movies",
    "posters",
    "releases",
    "studios",
    "themes",
    "rotten_tomatoes_reviews",
    "oscar_awards",
]

def clear_tables():
    """Cancella tutte le entry dalle tabelle del database."""
    try:
        # Connessione al database
        conn = psycopg2.connect(**DB_CONFIG)
        conn.autocommit = True  # Attiva la modalità autocommit
        cursor = conn.cursor()

        print("Inizio cancellazione dei dati...")

        for table in TABLES:
            query = sql.SQL("TRUNCATE TABLE {} CASCADE").format(sql.Identifier(table))
            cursor.execute(query)
            print(f"Tabella '{table}' svuotata con successo.")

        print("Tutte le tabelle sono state svuotate correttamente.")

        cursor.close()
        conn.close()

    except Exception as e:
        print(f"Errore durante la cancellazione delle tabelle: {e}")

if __name__ == "__main__":
    clear_tables()


Inizio cancellazione dei dati...
Tabella 'actors' svuotata con successo.
Tabella 'countries' svuotata con successo.
Tabella 'crew' svuotata con successo.
Tabella 'genres' svuotata con successo.
Tabella 'languages' svuotata con successo.
Tabella 'movies' svuotata con successo.
Tabella 'posters' svuotata con successo.
Tabella 'releases' svuotata con successo.
Tabella 'studios' svuotata con successo.
Tabella 'themes' svuotata con successo.
Tabella 'rotten_tomatoes_reviews' svuotata con successo.
Tabella 'oscar_awards' svuotata con successo.
Tutte le tabelle sono state svuotate correttamente.


In [3]:
import psycopg2
from psycopg2 import sql
import csv
import io

# Configurazione della connessione al database
DB_CONFIG = {
    "dbname": "Movies_DB",
    "user": "postgres",
    "password": "Cetriolo20!",
    "host": "localhost",
    "port": "5432",
}

# Configurazione delle tabelle e relativi file CSV
CSV_FILES = [
    {"table": "actors", "file": "C:/Users/franc/PycharmProjects/Assignment 2024-2025/pythonProject/Data/actors.csv", "columns": ["id", "name", "role"]},
    {"table": "countries", "file": "C:/Users/franc/PycharmProjects/Assignment 2024-2025/pythonProject/Data/countries.csv", "columns": ["id", "country"]},
    {"table": "crew", "file": "C:/Users/franc/PycharmProjects/Assignment 2024-2025/pythonProject/Data/crew.csv", "columns": ["id", "role", "name"]},
    {"table": "genres", "file": "C:/Users/franc/PycharmProjects/Assignment 2024-2025/pythonProject/Data/genres.csv", "columns": ["id", "genre"]},
    {"table": "languages", "file": "C:/Users/franc/PycharmProjects/Assignment 2024-2025/pythonProject/Data/languages.csv", "columns": ["id", "type", "language"]},
    {"table": "movies", "file": "C:/Users/franc/PycharmProjects/Assignment 2024-2025/pythonProject/Data/movies.csv", "columns": ["id", "name", "date", "tagline", "description", "minute", "rating"]},
    {"table": "posters", "file": "C:/Users/franc/PycharmProjects/Assignment 2024-2025/pythonProject/Data/posters.csv", "columns": ["id", "link"]},
    {"table": "releases", "file": "C:/Users/franc/PycharmProjects/Assignment 2024-2025/pythonProject/Data/releases.csv", "columns": ["id", "country", "date", "type", "rating"]},
    {"table": "studios", "file": "C:/Users/franc/PycharmProjects/Assignment 2024-2025/pythonProject/Data/studios.csv", "columns": ["id", "studio"]},
    {"table": "themes", "file": "C:/Users/franc/PycharmProjects/Assignment 2024-2025/pythonProject/Data/themes.csv", "columns": ["id", "theme"]},
    {"table": "oscar_awards", "file": "C:/Users/franc/PycharmProjects/Assignment 2024-2025/pythonProject/Data/the_oscar_awards.csv", "columns": ["year_film", "year_ceremony", "ceremony", "category", "name", "film", "winner"]},
    {"table": "rotten_tomatoes_reviews", "file": "C:/Users/franc/PycharmProjects/Assignment 2024-2025/pythonProject/Data/rotten_tomatoes_reviews.csv", "columns": ["rotten_tomatoes_link", "movie_title", "critic_name", "top_critic", "publisher_name", "review_type", "review_score", "review_date", "review_content"]},
]

# Funzione per rimuovere i vincoli di unicità
def remove_constraints(cursor):
    tables_to_update = ["actors", "countries", "crew", "genres", "languages", "posters", "releases", "studios", "themes"]
    for table in tables_to_update:
        constraint_name = f"{table}_pkey"
        try:
            cursor.execute(
                sql.SQL("ALTER TABLE {} DROP CONSTRAINT IF EXISTS {};")
                .format(sql.Identifier(table), sql.Identifier(constraint_name))
            )
            print(f"Vincolo di unicità rimosso da '{table}'.")
        except Exception as e:
            print(f"Errore durante la rimozione del vincolo di unicità per '{table}': {e}")

def filter_and_import_csv(cursor, table_name, file_path, columns):
    try:
        if table_name == "rotten_tomatoes_reviews":
            normalized_data = io.StringIO()
            with open(file_path, "r", encoding="utf-8") as file:
                reader = csv.DictReader(file)
                writer = csv.DictWriter(normalized_data, fieldnames=columns)
                writer.writeheader()

                for row in reader:
                    score = row["review_score"]
                    try:
                        # Convertire '3.5/5' in un valore numerico
                        if "/" in score:
                            numerator, denominator = map(float, score.split("/"))
                            if denominator == 0:  # Evitare divisioni per zero
                                row["review_score"] = None
                            else:
                                row["review_score"] = numerator / denominator
                        else:
                            row["review_score"] = float(score)
                    except (ValueError, TypeError):
                        row["review_score"] = None
                    writer.writerow(row)

            normalized_data.seek(0)
            file_to_import = normalized_data
        elif table_name == "posters":
            filtered_data = io.StringIO()
            with open(file_path, "r", encoding="utf-8") as file:
                reader = csv.DictReader(file)
                writer = csv.DictWriter(filtered_data, fieldnames=columns)
                writer.writeheader()
                for row in reader:
                    if row["link"] and row["link"].strip().lower() != "null":
                        writer.writerow(row)
            filtered_data.seek(0)
            file_to_import = filtered_data
        else:
            file_to_import = open(file_path, "r", encoding="utf-8")

        # Importazione dei dati
        cursor.copy_expert(
            sql.SQL("""
                COPY {} ({}) FROM STDIN WITH (FORMAT CSV, HEADER TRUE, QUOTE '"', NULL '')
            """).format(
                sql.Identifier(table_name),
                sql.SQL(", ").join(map(sql.Identifier, columns))
            ),
            file_to_import
        )
        print(f"Dati importati con successo nella tabella '{table_name}'.")
    except Exception as e:
        print(f"Errore durante l'importazione nella tabella '{table_name}': {e}")
    finally:
        if table_name in ["rotten_tomatoes_reviews", "posters"]:
            file_to_import.close()
        else:
            file_to_import.close()

# Funzione principale
def main():
    try:
        # Connessione al database
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()

        # Rimuove vincoli di unicità per le tabelle escluse
        remove_constraints(cursor)

        for csv_file in CSV_FILES:
            table_name = csv_file["table"]
            file_path = csv_file["file"]
            columns = csv_file["columns"]
            filter_and_import_csv(cursor, table_name, file_path, columns)

        conn.commit()
        print("Tutti i dati sono stati importati con successo.")
    except Exception as e:
        print(f"Errore durante il processo di importazione: {e}")
    finally:
        if 'cursor' in locals():
            cursor.close()
        if 'conn' in locals():
            conn.close()

if __name__ == "__main__":
    main()


Vincolo di unicità rimosso da 'actors'.
Vincolo di unicità rimosso da 'countries'.
Vincolo di unicità rimosso da 'crew'.
Vincolo di unicità rimosso da 'genres'.
Vincolo di unicità rimosso da 'languages'.
Vincolo di unicità rimosso da 'posters'.
Vincolo di unicità rimosso da 'releases'.
Vincolo di unicità rimosso da 'studios'.
Vincolo di unicità rimosso da 'themes'.
Dati importati con successo nella tabella 'actors'.
Dati importati con successo nella tabella 'countries'.
Dati importati con successo nella tabella 'crew'.
Dati importati con successo nella tabella 'genres'.
Dati importati con successo nella tabella 'languages'.
Dati importati con successo nella tabella 'movies'.
Dati importati con successo nella tabella 'posters'.
Dati importati con successo nella tabella 'releases'.
Dati importati con successo nella tabella 'studios'.
Dati importati con successo nella tabella 'themes'.
Dati importati con successo nella tabella 'oscar_awards'.
Dati importati con successo nella tabella 'rot

In [4]:
import psycopg2

# Dettagli di connessione al db
host = 'localhost' 
dbname = 'Movies_DB'
user = 'postgres'
password = 'Cetriolo20!'

# Connessione al database
conn = psycopg2.connect(host=host, dbname=dbname, user=user, password=password)
cur = conn.cursor()

# Query per ottenere il nome di tutte le tabelle nel database
cur.execute("""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public'
""")

tables = cur.fetchall()

for table in tables:
    table_name = table[0]
    cur.execute(f"SELECT COUNT(*) FROM {table_name}")
    row_count = cur.fetchone()[0]
    print(f"Tabella {table_name} contiene {row_count} righe.")

# Chiude connessione
cur.close()
conn.close()


Tabella countries contiene 693476 righe.
Tabella actors contiene 5798450 righe.
Tabella crew contiene 4720183 righe.
Tabella genres contiene 1046849 righe.
Tabella languages contiene 1038762 righe.
Tabella movies contiene 941597 righe.
Tabella posters contiene 760885 righe.
Tabella releases contiene 1332782 righe.
Tabella studios contiene 679283 righe.
Tabella themes contiene 125641 righe.
Tabella rotten_tomatoes_reviews contiene 1129887 righe.
Tabella oscar_awards contiene 10889 righe.


In [5]:
import csv

def count_rows_in_csv(file_path):
    try:
        with open(file_path, newline='', encoding='utf-8') as csvfile:
            csvreader = csv.reader(csvfile)
            row_count = sum(1 for row in csvreader)  # Conta tutte le righe
        return row_count
    except FileNotFoundError:
        print(f"File not trovato: {file_path}")
        return None
    except Exception as e:
        print(f"Errore: {e}")
        return None

file_path = 'C:/Users/franc/PycharmProjects/Assignment 2024-2025/pythonProject/Data/actors.csv'
num_rows = count_rows_in_csv(file_path)

if num_rows is not None:
    print(f"Il file CSV contiene {num_rows} righe.")


Il file CSV contiene 5798451 righe.
